In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re, time, json, openpyxl
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [18]:
%%time
df = pd.DataFrame(columns=['Период', 'Регион', 'Код товара (услуги)', 'Наименование товара (услуги)', 'Цена'])

date_start = datetime(2024, 1, 1) #.strftime("%Y-%m-%d")
# date_end = datetime(2024, 8 , 21) #.strftime("%Y-%m-%d")
date_end = datetime.today()
date_df = pd.date_range(date_start, date_end)

region_name = []
product_name = []
product_code = []
price = []
period = []

week = 1
# now_date = start

for dat in tqdm(date_df):

    period_intermediate = []

    url = f'https://showdata.gks.ru/x/report/274422/view/compound/?filter_1_0={dat.strftime("%Y-%m-%d")}+00%3A00%3A00%7C-58&filter_2_0=109543%2C109422%2C109423%2C109425%2C109517%2C109549%2C109427%2C109550%2C109428%2C109430%2C339613%2C109518%2C109396%2C109397%2C109432%2C347487%2C109435%2C109560%2C109561%2C109562%2C109563%2C109566%2C109444%2C109451%2C109577%2C109524%2C109402%2C109525%2C109526%2C109457%2C109459%2C109460%2C339606%2C109463%2C109464%2C109466%2C109468%2C109469%2C109470%2C109471%2C109594%2C109472%2C109595%2C109474%2C109597%2C109476%2C109479%2C109726%2C110321%2C347014%2C109530%2C217794%2C109744%2C109534%2C109652%2C109654%2C109781%2C109681%2C109688%2C109811%2C109692%2C109693%2C109816%2C109831%2C109715%2C339611%2C109841%2C109842%2C109721%2C109723%2C109849%2C109854%2C110387%2C339616%2C110389%2C109972%2C109900%2C109983%2C109901%2C318647%2C109996%2C109418%2C110335%2C109931%2C110018%2C110022%2C109944%2C109946%2C217805%2C110029%2C109947%2C110030%2C109948%2C110031%2C217807%2C217787%2C110038%2C110043%2C109961%2C109962%2C110045%2C110370%2C110174%2C110175%2C110053%2C110057%2C110180%2C110181%2C110184%2C110185%2C110063%2C110065%2C110188%2C109419%2C339627%2C339641%2C339632%2C339772%2C347492%2C110077%2C110086%2C110087%2C347477%2C110438%2C110102%2C110225%2C110103%2C110226%2C217806%2C110231%2C110252%2C110254%2C110135%2C110259%2C110137%2C110138%2C110261%2C110264%2C110142%2C110346%2C318633%2C318634%2C318614%2C318619%2C318622%2C110268%2C110278%2C110281%2C110416%2C110162%2C110285&filter_3_0=13062&rp_submit=t&_=1723879556931'
    response = requests.get(url, verify=False).json()
    if len(response['headers']['rowHeader']) != 0:
        # print(f'Обрабатывается неделя {week}!')
        # period_intermediate.append(dat.strftime("%Y-%m-%d"))
        
        for value in response['data']['reportData']['data']:
            price.append(value[0]['db_value'])
            period.append(dat)
            # for period_ in response['headers']['params']['conditions']:
            #     if period_['fieldName'] == 'filter_1_0':
            #         period.append(period_['value'])
                    
        for item in response['headers']['reportHeaders']['row_header']:
            for child in item['children']:
                region_name.append(child['display_title'])
    
        for item in response['headers']['reportHeaders']['row_header']: 
            product_code.append(item['extra_row_attrs']['code'])
            product_name.append(item['display_title']) 
        
        # print(f'Неделя {week} обработана!')
        week += 1
        print()

#         now_date = now_date + relativedelta(days=1)
            
    # except:
#         now_date = now_date + relativedelta(days=1)
        # continue
    
df['Период'] = period
df['Регион'] = region_name
df['Код товара (услуги)'] = product_code
df['Наименование товара (услуги)'] = product_name
df['Цена'] = price

df['Регион'] = df['Регион'].str.lstrip('0123456789')
df['Наименование товара (услуги)'] = df['Наименование товара (услуги)'].str.lstrip('0123456789')
df

  3%|▎         | 8/234 [00:03<01:14,  3.05it/s]

  6%|▋         | 15/234 [00:05<01:27,  2.50it/s]

  9%|▉         | 22/234 [00:08<01:08,  3.09it/s]

 12%|█▏        | 29/234 [00:10<01:06,  3.09it/s]

 15%|█▌        | 36/234 [00:12<01:05,  3.04it/s]

...

 96%|█████████▌| 225/234 [01:49<00:02,  3.10it/s]

 99%|█████████▉| 232/234 [01:51<00:00,  3.10it/s]

100%|██████████| 234/234 [01:52<00:00,  2.08it/s]

CPU times: user 3.42 s, sys: 403 ms, total: 3.82 s
Wall time: 1min 52s


,Период,Регион,Код товара (услуги),Наименование товара (услуги),Цена
0,2024-01-08,Ставропольский край,1001,"Маргарин, кг",248.17
1,2024-01-08,Ставропольский край,1102,"Сметана, кг",295.40
2,2024-01-08,Ставропольский край,111,"Говядина (кроме бескостного мяса), кг",543.56
3,2024-01-08,Ставропольский край,1111,"Молоко питьевое цельное пастеризованное 2,5-3...",78.07
4,2024-01-08,Ставропольский край,1112,"Молоко питьевое цельное стерилизованное 2,5-3...",85.25
...,...,...,...,...,...
3658,2024-08-19,Ставропольский край,9465,"Проживание в гостинице 4*-5*, сутки с человека",3050.29
3659,2024-08-19,Ставропольский край,9466,"Проживание в хостеле, сутки с человека",788.61
3660,2024-08-19,Ставропольский край,9475,Услуги по снабжению электроэнергией,601.80
3661,2024-08-19,Ставропольский край,9606,"Санаторий, день",4894.40


In [ ]:
df.to_excel('Test.xlsx')